In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
warnings.filterwarnings("ignore",".*GUI is implemented.*")

In [4]:
from importlib import reload
import lib2

In [5]:
from time import sleep

In [6]:
from lib import data_management as dm

In [7]:
import drivers

In [8]:
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import *

In [9]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("TEST", -6))
ro_cal = calibrations.get(frozenset(dict(lo_power=0, ssb_power=-30, lo_frequency=7.047e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("TESTQ", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-10, lo_frequency=7.9e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
q_pb = PulseBuilder(q_cal)

In [114]:
ro_awg = KeysightAWG("AWG3")

In [115]:
q_awg = KeysightAWG("AWG2")

In [46]:
from drivers.Keysight_DSOX2014 import *

In [48]:
dso = Keysight_DSOX2014("DSO2014")

# Spectroscopy

## Single-tone spectroscopy

In [7]:
# reload(lib2.Measurement)

In [10]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [55]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100, 5).build())

In [66]:
STS = SingleToneSpectroscopy("VI-anticrossing", "Xmon Al BMSTU S444 2", vna = 'vna2', src = 'yok3')
vna_parameters = {"bandwidth":500, "freq_limits":(7.04e9, 7.06e9), "nop":201, "power":-20, "averages":1}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(-0.9e-3, 0.75e-3, 100)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);

vna2 is already initialized
yok3 is already initialized
Started at:  2017-05-10 19:40:54.585609


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, current: 7.50e-04]: , average cycle time: 0.47 s          

In [65]:
result.visualize()

In [44]:
result.save()

## Two-tone spectroscopy

In [34]:
# reload(lib2.Measurement)
# reload(lib2.TwoToneSpectroscopyBase)
# reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [54]:
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100,5).build())

#### Qubit spectrum

In [14]:
Measurement._actual_devices.keys()

dict_keys(['vna3', 'yok4'])

In [81]:
TTS = FluxTwoToneSpectroscopy("VI-two-tone", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":50, "freq_limits":(7.04e9, 7.06e9), "nop":1, "power":-20, "averages":1}
mw_src_parameters = {"power":0}
mw_src_frequencies = linspace(7.8e9, 7.95e9, 50)
currents = linspace(-0.4e-3, 0.2e-3, 50)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7060000000.0)                    at qubit's sweet spot (-0.10 mA)
Detected frequency is 7.04673 GHz, at 0.70 mU and 5.02 degrees


In [82]:
result = TTS.launch()

Started at:  2017-05-10 19:57:51.319763
Time left: 0 h 3 m 21.59 s, parameter value: -4.000e-04, average cycle time: 0.08 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 2.000e-04, average cycle time: 0.06 s             

In [43]:
result.visualize()

In [83]:
result.save()

In [20]:
result._name = "5.5GHz-two-tone-w-loop_-0.5mA-and-coil"

In [46]:
TTS._mw_src.set_output_state("OFF")

#### Linewidth from excitation power

In [185]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [183]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":0, "averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-20, 15, 21)
current = -0.11e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.10 mU and 33.83 degrees


In [187]:
result = PTS.launch()

Started at:  2017-05-10 21:15:08.685005
Time left: 0 h 10 m 10.55 s, parameter value: -2.000e+01, average cycle time: 0.15 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 9 m 25.8 s, parameter value: -2.000e+01, average cycle time: 0.14 s           

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [106]:
result.visualize()

In [103]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [206]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [208]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-10, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.85e9, 7.9e9, 100)
vna_powers = linspace(-30, 0, 31)
current = -0.11e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.25 mU and 33.57 degrees


In [21]:
result = ASTS.launch()

Started at:  2017-04-25 22:15:13.034443
Time left: 0 h 27 m 19.46 s, parameter value: -3.000e+01, average cycle time: 0.53 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.16 s              

In [112]:
result.visualize()

In [12]:
result.save()

In [88]:
TTS.close_devs("mxg")

__________________________

# Time-domain measurements

#### Check mixers:

In [175]:
q_lo = mxg
q_lo.set_output_state("ON")
q_lo.set_power(15)

In [96]:
TTS._vna.set_nop(1)
TTS._vna.set_center(7.04720e9)
TTS._vna.sweep_continuous()

In [109]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [150]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(20,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(20).build())

In [170]:
print(ro_cal)

Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 0, 'ssb_power': -40, 'waveform_resolution': 1}
Optimization results: {'dc': -87.92120361328125, 'dc_open': [-40.00850296020508]}
Optimization parameters {'dc_offsets': array([ 0.00884471, -0.06675995]), 'dc_offsets_open': array([ 0.10280304,  0.1024881 ]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 13.31 s
Finished at: 2017-05-10 21:07:10.849674


### Rabi 

In [10]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [14]:
DRO = DispersiveRabiOscillations("VI-rabi", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
current = -0.11e-3
q_frequency = 7.897e9
excitation_durations = linspace(1, 150, 38)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":8000}
DRO.set_fixed_parameters(vna_parameters, ro_cal, q_cal, q_frequency, current, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04710 GHz, at 0.04 mU and 13.13 degrees


In [15]:
result = DRO.launch()

Started at:  2017-05-12 12:53:23.876306
Time left: 0 h 1 m 39.23 s, [excitation_duration: 1.71e+01]: , average cycle time: 3.01 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [excitation_duration: 1.50e+02]: , average cycle time: 3.06 s         

In [18]:
result.visualize()

(<matplotlib.figure.Figure at 0xf00dfd0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000B47C2E8>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000B42D4E0>], dtype=object),
 (None, None))

In [22]:
result.save()

In [43]:
from lib2.MeasurementResult import *
result = MeasurementResult.load("Xmon Al BMSTU S444 2","VI-ramsey")

In [44]:
result.visualize()

(<matplotlib.figure.Figure at 0x9dfea58>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000001180D9B0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000009D7C4A8>], dtype=object),
 (None, None))

In [16]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

74.508759194533937

### Decay

In [17]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [19]:
DD = DispersiveDecay("VI-decay", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
current = -0.11e-3
q_frequency = 7.897e9
readout_delays = linspace(0, 1, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "pi_pulse_duration":pi_pulse_duration}
DD.set_fixed_parameters(vna_parameters, ro_cal, q_cal, q_frequency, current, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04710 GHz, at 0.04 mU and 11.61 degrees


In [20]:
dd_result = DD.launch()

Started at:  2017-05-12 12:55:52.840826
Time left: 0 h 4 m 43.46 s, [readout_delay: 4.04e-02]: , average cycle time: 2.98 s       

In [55]:
dd_result.visualize()

(<matplotlib.figure.Figure at 0x11283b70>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000E3C70B8>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000F228DA0>], dtype=object),
 (None, None))

In [28]:
dd_result.save()

Time left: 0 h 7 m 13.22 s, [readout_delay: 2.83e+03]: , average cycle time: 10.07 s       

### Ramsey

In [42]:
reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [34]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
current = -0.11e-3
q_frequency = 7.897e9
readout_delays = linspace(0, 2000, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
DR.set_fixed_parameters(vna_parameters, ro_cal, q_cal, q_frequency, current, rabi_sequence_parameters)
DR.set_swept_parameters(readout_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.04 mU and 14.64 degrees


In [35]:
dr_result = DR.launch()

Started at:  2017-05-12 13:36:29.933220
Time left: 0 h 4 m 43.55 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.98 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [ramsey_delay: 2.00e+03]: , average cycle time: 3.06 s         

In [38]:
dr_result.visualize()

(<matplotlib.figure.Figure at 0xe15d668>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000E0EAE10>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x00000000118679E8>], dtype=object),
 (None, None))

In [37]:
dr_result.save()

In [161]:
q_pb = PulseBuilder(q_cal)

In [39]:
pi_pulse_duration/2

38.552901565627764

In [130]:
ramsey_delay = 0
half_pi_pulse_duration = pi_pulse_duration/2
ramsey_seq = q_pb.add_zero_pulse(1)\
            .add_sine_pulse(half_pi_pulse_duration, 0)\
            .add_zero_pulse(ramsey_delay)\
            .add_sine_pulse(half_pi_pulse_duration, time_offset = half_pi_pulse_duration+ramsey_delay)\
            .add_zero_until(200).build()
decay_seq = q_pb.add_zero_pulse(1)\
            .add_sine_pulse(2*half_pi_pulse_duration, 0).add_zero_until(200).build()\
            
#         self._q_awg.output_pulse_sequence(self._q_pb.build())

In [131]:
round(half_pi_pulse_duration/0.1)

386.0

In [132]:
round(half_pi_pulse_duration/0.1)

386.0

In [133]:
ramsey_seq.plot()
decay_seq.plot()
plt.grid()

In [129]:
len(ramsey_seq.get_I_waveform())

2001

In [136]:
DR._q_awg.output_pulse_sequence(decay_seq)

In [156]:
round(pi_pulse_duration/2/0.1), pi_pulse_duration/2

(339.0, 33.852893886313765)

In [153]:
round((2000-1-1000-pi_pulse_duration-400)/0.1)/10+1000+pi_pulse_duration+1+400

2000.0057877726274

In [53]:
dso.set_time_range(1e-6)
dso.set_time_offset(.1e-6)
dso.digitize()
times, channel_data = dso.get_data(Channel.ONE, Channel.TWO)
axes = flatten(plt.subplots(2,1, sharex=True, figsize=(13,5))[1])
colors = ["C0", "C1", "C2", "C3"]
for idx, ax in enumerate(axes):
    ax.plot(times, channel_data[idx], color=colors[idx])
    ax.minorticks_on()
    ax.grid(which="both")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.tight_layout()